- This notebook to get latent space of surfactants dataset by using chemprop

In [9]:
import os
import torch
import chemprop
from chemprop import data, featurizers, models
from chemprop import featurizers, nn

In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [24]:
checkpoint_path='../src/models/chemprop/example_model_v2_regression_mol.ckpt'
mpnn = models.MPNN.load_from_checkpoint(checkpoint_path)
mpnn.eval()
mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=372, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): GraphTransform(
      (V_transform): Identity()
      (E_transform): Identity()
    )
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=300, out_features=300, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=300, out_features=1, bias=True)
      )
    )
    (criterion): MSE(task_weights=[[1.0]])
    (output_transform): UnscaleTransform()
  )
  (X_d_transform): Identity()
  (metric

In [4]:
# Get the data

In [ ]:
file_path = '../../data/Literature/surfactant_paper/surfpro_literature.csv'
df_data = pd.read_csv(file_path)
df_data

,SMILES,Surfactant_Type,CMC,pCMC,AW_ST_CMC,Gamma_max,Area_min,Pi_CMC,pC20,C20,...,SubRef_AW_ST_CMC,Ref_Gamma_max,Ref_Pi_CMC,Ref_Area_min,Ref_pC20,Ref_C20,IUPAC_name,InChI,InChIKey,Molecular_Weight
0,C(=Nc1cc[n+](CCCCCCCCCCCC[n+]2ccc(N=Cc3ccccc3)...,gemini cationic,NaN,NaN,41.0,NaN,NaN,NaN,NaN,NaN,...,10.1007/s11743-013-1478-9,NaN,NaN,NaN,NaN,NaN,NaN,InChI=1S/C36H44N4.2BrH/c1(3-5-7-15-25-39-27-21...,DPTDXTODPMJJNL-UHFFFAOYSA-L,690.193272
1,C(=Nc1cc[n+](CCCCCC[n+]2ccc(N=Cc3ccccc3)cc2)cc...,gemini cationic,NaN,NaN,47.0,NaN,NaN,NaN,NaN,NaN,...,10.1007/s11743-013-1478-9,NaN,NaN,NaN,NaN,NaN,NaN,InChI=1S/C30H32N4.2BrH/c1(9-19-33-21-15-29(16-...,AHXCMXJBNCLAHK-UHFFFAOYSA-L,606.099371
2,C(=Nc1cc[n+](CC[n+]2ccc(N=Cc3ccccc3)cc2)cc1)c1...,gemini cationic,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,...,10.1007/s11743-013-1478-9,NaN,NaN,NaN,NaN,NaN,NaN,InChI=1S/C26H24N4.2BrH/c1-3-7-23(8-4-1)21-27-2...,MVHGWIJZXNNERZ-UHFFFAOYSA-L,550.036771
3,C=CC(=O)OC(C[N+](C)(C)CCCCCCCCCCCC)C[N+](C)(C)...,gemini cationic,0.000790,3.102373,33.4,0.000001,1.12,38.6,3.836155,0.021576,...,10.1016/j.colsurfa.2013.12.079,10.1016/j.colsurfa.2013.12.079,NaN,10.1016/j.colsurfa.2013.12.079,calculated,calculated,"1,3-Bis(N,N-dimethyl-N-dodecylammonium)-2-prop...",InChI=1S/C34H70N2O2.2BrH/c1-8-11-13-15-17-19-2...,ARKWXUQYSTUBMC-UHFFFAOYSA-L,696.380404
4,C=CC(=O)OC(C[N+](C)(C)CCCCCCCCCCCCCC)C[N+](C)(...,gemini cationic,0.000240,3.619789,31.3,0.000001,1.19,40.7,4.483082,0.011299,...,10.1016/j.colsurfa.2013.12.079,10.1016/j.colsurfa.2013.12.079,NaN,10.1016/j.colsurfa.2013.12.079,calculated,calculated,"1,3-Bis(N,N-dimethyl-N-tetradecylammonium)-2-p...",InChI=1S/C38H78N2O2.2BrH/c1-8-11-13-15-17-19-2...,VTTIVKLPSBNYRL-UHFFFAOYSA-L,752.443004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,OCCOCCOCCOCCOCCOCCSCCC(F)(F)C(F)(F)C(F)(F)C(F)...,non-ionic,0.000037,4.432033,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"InChI=1S/C20H29F13O6S/c21-15(22,16(23,24)17(25...",VOCCWGUEVZCMEP-UHFFFAOYSA-N,644.147727
1620,OCCOCCOCCOCCSCCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F...,non-ionic,0.000028,4.552997,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"2-[2-[2-[2-(3,3,4,4,5,5,6,6,7,7,8,8,8-tridecaf...","InChI=1S/C16H21F13O4S/c17-11(18,1-9-34-10-8-33...",HVOWVEVSZCKCCE-UHFFFAOYSA-N,556.095297
1621,OCCOCCOCCSCCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F...,non-ionic,0.000025,4.602060,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"2-[2-[2-(3,3,4,4,5,5,6,6,7,7,8,8,8-tridecafluo...","InChI=1S/C14H17F13O3S/c15-9(16,1-7-31-8-6-30-5...",IOGSRXKLAFCVIR-UHFFFAOYSA-N,512.069082
1622,OCCOCCSCCOCCSCCOCCSCCC(F)(F)C(F)(F)C(F)(F)C(F)...,non-ionic,0.000026,4.585027,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"InChI=1S/C20H29F13O4S3/c21-15(22,16(23,24)17(2...",LPHAEYJKVNLDKF-UHFFFAOYSA-N,676.102039


In [15]:
df_ = df_data.iloc[:, [0, 1, 3]]
df_cleaned = df_.dropna()
df_data_clean = df_cleaned.reset_index(drop=True)
df_data_clean

,SMILES,Surfactant_Type,pCMC
0,C=CC(=O)OC(C[N+](C)(C)CCCCCCCCCCCC)C[N+](C)(C)...,gemini cationic,3.102373
1,C=CC(=O)OC(C[N+](C)(C)CCCCCCCCCCCCCC)C[N+](C)(...,gemini cationic,3.619789
2,C=CC(=O)OC(C[N+](C)(C)CCCCCCCCCCCCCCCC)C[N+](C...,gemini cationic,4.508638
3,C=CC(=O)OCC[N+](C)(C)CCCCCCCCCCCC.[Br-],cationic,2.384050
4,C=CC(=O)OCC[N+](C)(C)CCCCCCCCCCCCCC.[Br-],cationic,2.899629
...,...,...,...
1390,OCCOCCOCCOCCOCCOCCSCCC(F)(F)C(F)(F)C(F)(F)C(F)...,non-ionic,4.432033
1391,OCCOCCOCCOCCSCCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F...,non-ionic,4.552997
1392,OCCOCCOCCSCCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F...,non-ionic,4.602060
1393,OCCOCCSCCOCCSCCOCCSCCC(F)(F)C(F)(F)C(F)(F)C(F)...,non-ionic,4.585027


In [16]:
smis = df_data_clean['SMILES']
test_data = [data.MoleculeDatapoint.from_smi(smi) for smi in smis]
test_data[:5]

[MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x150954ba0>, y=None, weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='C=CC(=O)OC(C[N+](C)(C)CCCCCCCCCCCC)C[N+](C)(C)CCCCCCCCCCCC.[Br-].[Br-]', V_f=None, E_f=None, V_d=None),
 MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x1509552a0>, y=None, weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='C=CC(=O)OC(C[N+](C)(C)CCCCCCCCCCCCCC)C[N+](C)(C)CCCCCCCCCCCCCC.[Br-].[Br-]', V_f=None, E_f=None, V_d=None),
 MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x150954cf0>, y=None, weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='C=CC(=O)OC(C[N+](C)(C)CCCCCCCCCCCCCCCC)C[N+](C)(C)CCCCCCCCCCCCCCCC.[Br-].[Br-]', V_f=None, E_f=None, V_d=None),
 MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x150954740>, y=None, weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='C=CC(=O)OCC[N+](C)(C)CCCCCCCCCCCC.[Br-]', V_f=None, E_f=None, V_d=None),
 MoleculeDa

In [18]:
test_dset = data.MoleculeDataset(test_data, featurizer=featurizer)
test_loader = data.build_dataloader(test_dset, shuffle=False)

In [25]:
with torch.no_grad():
    encodings = [
        mpnn.encoding(batch.bmg, batch.V_d, batch.X_d, i=-1)
        for batch in test_loader
    ]
    encodings = torch.cat(encodings, 0)

encodings.shape

torch.Size([1395, 300])

In [27]:
len(test_data)

1395

In [70]:
encodings.detach().shape

torch.Size([1395, 300])

In [55]:
encodings_df = pd.DataFrame(encodings.numpy()) 
df_total = pd.concat([encodings_df, df_data_clean[['Surfactant_Type', 'pCMC']]], axis=1)
df_total

,0,1,2,3,4,5,6,7,8,9,...,292,293,294,295,296,297,298,299,Surfactant_Type,pCMC
0,-0.000767,-0.032740,0.012556,0.041580,-0.042636,0.043729,0.036568,-0.026811,-0.099208,-0.029661,...,0.041115,-0.010880,-0.064434,0.133297,0.048652,-0.116293,0.028970,0.027461,gemini cationic,3.102373
1,0.000664,-0.032233,0.013568,0.041165,-0.043851,0.042598,0.036527,-0.027357,-0.101479,-0.028759,...,0.041094,-0.011109,-0.063384,0.134593,0.048573,-0.118641,0.029983,0.027760,gemini cationic,3.619789
2,0.001856,-0.031811,0.014412,0.040819,-0.044864,0.041654,0.036494,-0.027812,-0.103372,-0.028008,...,0.041076,-0.011300,-0.062509,0.135673,0.048507,-0.120598,0.030827,0.028009,gemini cationic,4.508638
3,0.003190,-0.034874,0.015333,0.042935,-0.043670,0.039052,0.035544,-0.026676,-0.101058,-0.034714,...,0.041076,-0.008852,-0.066290,0.129257,0.045773,-0.106820,0.027165,0.020545,cationic,2.384050
4,0.004132,-0.034258,0.016002,0.042461,-0.044656,0.038430,0.035591,-0.027167,-0.102909,-0.033516,...,0.041061,-0.009215,-0.065218,0.130720,0.045934,-0.109644,0.028201,0.021361,cationic,2.899629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390,-0.018337,-0.048820,-0.016457,0.027891,-0.030037,0.041121,0.058026,-0.005086,-0.099076,-0.047239,...,0.028756,0.013613,-0.078583,0.106162,0.065398,-0.097669,0.051315,0.011931,non-ionic,4.432033
1391,-0.023282,-0.049368,-0.018658,0.030324,-0.028702,0.041561,0.059125,-0.002364,-0.101201,-0.048764,...,0.029247,0.015237,-0.083964,0.099190,0.065856,-0.093342,0.052581,0.007416,non-ionic,4.552997
1392,-0.026473,-0.049721,-0.020079,0.031893,-0.027840,0.041845,0.059834,-0.000608,-0.102572,-0.049749,...,0.029564,0.016284,-0.087437,0.094691,0.066152,-0.090550,0.053398,0.004504,non-ionic,4.602060
1393,-0.020579,-0.047691,-0.017068,0.031032,-0.028610,0.041001,0.055560,-0.006995,-0.098446,-0.049058,...,0.028404,0.012521,-0.079723,0.102980,0.063607,-0.098309,0.052664,0.012853,non-ionic,4.585027


In [73]:
df_total.to_csv('df_total_surfactants_chemprop_20250707.csv', index=False)

### Binary case

In [77]:
os.chdir('/Users/haoliu/Documents/GitHub/FoundationModel_on_Liposome/code/notebooks')

In [78]:
pwd

'/Users/haoliu/Documents/GitHub/FoundationModel_on_Liposome/code/notebooks'

In [79]:
import sys
sys.path.append("/Users/haoliu/chemprop")
sys.path.append("../src/models")
sys.path.append("../src")
sys.path.append("../")

In [80]:
from chemeleon import build_mixture_latent_features

In [81]:
def normalize_concentrations(df, conc_cols):
    conc_sum = df[conc_cols].sum(axis=1)
    for conc in conc_cols:
        df[conc] = df[conc] / conc_sum
    return df

In [ ]:
df_mixture = pd.read_csv('../../data/Intermediate/df_mixture_20250502.csv')
df_mixture.head()
# Normalize concentrations
conc_cols = [f'conc{i}' for i in range(1, 3)]
df_mixture_norm = normalize_concentrations(df_mixture, conc_cols)
df_mixture_norm.head()

,smi1,conc1,smi2,conc2,miscibility
0,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,0.407074,CC(C)(C)CC(C)(C)C1=CC=C(C=C1)OCCO,0.592926,1
1,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,0.784804,[C@H]1([C@@H]([C@@H](OC([C@H]1O)C(=O)O)O[C@H]2...,0.215196,1
2,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,0.729393,C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)O[C@@H]2[...,0.270607,1
3,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,0.953164,C1[C@@H]2[C@@H]([C@H](O1)[C@H]([C@H](O2)O)OS(=...,0.046836,1
4,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,0.962481,C1[C@@H]2[C@@H]([C@H](O1)[C@H]([C@H](O2)O)O)O[...,0.037519,1
